# Исследование объявлений о продаже квартир

В вашем распоряжении данные сервиса Яндекс.Недвижимость — архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктов за несколько лет. Нужно научиться определять рыночную стоимость объектов недвижимости. Ваша задача — установить параметры. Это позволит построить автоматизированную систему: она отследит аномалии и мошенническую деятельность. 

По каждой квартире на продажу доступны два вида данных. Первые вписаны пользователем, вторые — получены автоматически на основе картографических данных. Например, расстояние до центра, аэропорта, ближайшего парка и водоёма. 

**Цель исследования состоит в следующем:**
* Найти параметры, по которым определяется цена недвижимости, чтобы система смогла следить за аномалиями и определять мошенническую деятельность
* Поиск особенностей и зависимостей, котрые есть на рынке недвижимости

**Ход исследования:**
* Изучение общей информации датасета
* Поиск и замена пропущенных значений, изменение типа данных, поиск и обработка аномалий
* Добавление новых столбцов с данными для более полного анализа
* Изучение параметров объектов недвижимости, исследование скорости продажи объектов, иучение зависимости цены от различных параметров объектов
* Определение самой высокой и самой низкой стоимости квадратного метра
* Определение зависимости цены объекта от удалённости от центра

### Откройте файл с данными и изучите общую информацию. 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
data = pd.read_csv('/datasets/real_estate_data.csv', sep='\t')
#Посмотрим наличие пропусков
data.info()
#Посмотрим общую информацию
data.describe()

In [ ]:
#Выведем первые 10 строк датасета
data.head(10)

In [ ]:
#Посмотрим количество пропусков
data.isna().sum()

In [ ]:
#Построим гистограмму для всех столбцов
import matplotlib.pyplot as plt
data.hist(grid=True, figsize=(15, 20));plt.show()

**Пройдёмся по гистограммам:**
В целом значения распределяются нормально. Выбросов не наблюдается.

**Вывод:**
В столбцах ceiling_height, floors_total, living_area, is_apartment, kitchen_area, balcony, locality_name, airports_nearest, cityCenters_nearest, parks_around3000, parks_nearest, ponds_around3000, ponds_nearest, days_exposition есть пропуски.
Также:
1. У столбца first_day_exposition долен быть тип "datetime"
2. У столбцов floors_total, balcony, parks_around3000, ponds_around3000, days_exposition, airports_nearest, cityCenters_nearest, parks_nearest, ponds_nearest должен быть тип "int", т.к. числовые значения в них могут быть только целыми (вряд ли кто-то решит указать цену с копейками)
3. У столбца is_apartment должен быть тип "bool"

Данные нужно привести в порядок.

* В столбце **is_apartment** слишком много пропусков, возможно это связано с тем, что многие люди не знают, что такое апартаменты, поэтому при заполнении данных они пропускали этот пункт. Знающие, наоборот, заполняли данные (раз они купили такую квартиру, значит разбираются в отличиях квартиры и апартаментов). Можно заменить NaN на "False", т.к., скорее всего, пропуски свидетельствуют о том, что квартира не является апартаментами (в договоре купли-продажи должна быть прописана информация о типе продаваемого/покупаемого помещения, в таком случае, если человек не знал, явлется ли квартира апартаментами, значит в договоре информаия про апартаменты не была указана, соответственно, помещение апартаментами не является).

* В столбце **balcony** пропуски можно заменить на "0", т.к., вероятно, что раз информация по наличию балкона не указана, то балкона нет.

* В столбце **ceiling_height** вместо пропусков можно поставить медианные значения. Пропуски могли появиться из-за того, что некоторые люди не считают эту информацию важно и обязательной к заполнению.

* Столбец **floors_total** оставим без изменений, данных для анализа досаточно. Причина может быть такой же, как и предыдущем пункте.

* Столбцы **living_area, kitchen_area** могут быть не заполнены из-за того, что квартира является студией. Пропуски заполнить не сможем (можно было бы рассчитать их путём вычитания и общей площади кухни/жилой площади, но у нас нет данных по площади сан. узла и коридора, которые также являются частью общей площади квартиры). Заполненных строк для анализа достаточно.

* Столбец **locality_name** оставим как есть. Непонятна причина пропусков.

* В столбцах **airports_nearest, cityCenters_nearest, parks_around3000, parks_nearest, ponds_around3000, ponds_nearest** данные были добавлены системой, мы их заполнить ничем не сможем, поэтому оставим всё как есть.

* Столбец **days_exposition** тоже оставим в покое. Медианные или средние значения не подойдут, т.к. длительность продажи может зависеть и от других факторов, по которым нет данных (например, шумные соседи, трубы текут, ремонта нет). Имеющихся данных для анализа достаточно. Пропуски могли быть из-за того, что продавцы могли не убрать объявление с сайта после продажи.

### Предобработка данных

In [ ]:
#Проверим % пропусков в датасете
pd.DataFrame(round(data.isna().mean()*100,)).style.background_gradient('coolwarm')

In [ ]:
#Посмотрим количество пропусков и уникальные значения в столбце balcony
data['balcony'].isna().sum()
data['balcony'].unique()
#Заменим пропуски в столбце balcony на "0" и заменим тип данных
data['balcony'] = data['balcony'].fillna(0).astype(int)

#Посмотрим количество пропусков и уникальные значения в столбце is_apartment
data['is_apartment'].isna().sum()
data['is_apartment'].unique()
#Заменим пропуски в столбце is_apartment на "False"
data['is_apartment'] = data['is_apartment'].fillna(False)

#Посмотрим количество пропусков и уникальные значения в столбце ceiling_height
data['ceiling_height'].isna().sum()
data['ceiling_height'].unique()
#Заменим пропуски в столбце ceiling_height на медианные значения
data['ceiling_height'] = data['ceiling_height'].fillna(data['ceiling_height'].median())

#Заменим тип данных в столбце first_day_exposition на datetime (время неизвестно, поэтому его указывать не будем)
data['first_day_exposition'] = pd.to_datetime(data['first_day_exposition'], format = '%Y-%m-%d')

#Заполним пропуски в столцах parks_around3000, ponds_around3000 нулями
data['parks_around3000'] = data['parks_around3000'].fillna(0)
data['ponds_around3000'] = data['ponds_around3000'].fillna(0)

#Заменим пропуски в столбце floors_total на значения столца floor той же строки
row = data['floor']
data['floors_total'] = data['floors_total'].fillna(row)

#Заменим тип данных в столбцах floors_total, parks_around3000, ponds_around3000, на int 
data['floors_total'] = data['floors_total'].astype(int)
data['parks_around3000'] = data['parks_around3000'].astype(int)
data['ponds_around3000'] = data['ponds_around3000'].astype(int)

#Тип столбцов days_exposition, airports_nearest, cityCenters_nearest, parks_nearest, ponds_nearest 
#не можем заменить на int из-за пропусков

#Проверим изменения
data.info()

In [ ]:
#Посмотрим значения столбца locality_name
data['locality_name'].value_counts()
data['locality_name'].unique()

In [ ]:
#Приведём названия к одному виду и удалим пропуски
bad_name = data[data['locality_name'].isnull()]
bad_ind = bad_name.index
print(bad_ind)
data.drop(bad_ind, inplace=True)

def short_name(row):
    split_name = row.split()
    new_name = []
    for name in split_name:
        if name[0].isupper():
            new_name.append(name)
    new_name = ''.join(name for name in new_name)
    return new_name

data['short_name'] = data['locality_name'].apply(short_name)
data['short_name'].unique()

In [ ]:
#Посмотрим уникальные значения столбца ceiling_height и выведем 30 последних значений
print(data['ceiling_height'].sort_values().tail(30))
data['ceiling_height'].unique()

Большие значения могли появиться из-за человеческого фактора. Продавец поставил запятую не там где надо, и вот у него в квартире уже 20-ти метровые потолки. А вот высота потолка в 8 метров непонятно откуда взялась, проблемя явно не в запятой, лучше заменим все большие значения на более реальные

In [ ]:
#Предположим, что 4-х метровые потолки действительно есть, в старых домах такая высота вполне может быть, а вот больше - вряд ли
#Сначала округлим значения до 1 знака после запятой
data['ceiling_height'] = data['ceiling_height'].round(1)
#Проверим преобразование
data['ceiling_height'].unique()
#Заменим значения больше 4.5 метров на 4.5
row = data['ceiling_height'] > 4.5
data.loc[row, ['ceiling_height']] = 4.5

#Проверим результат        
data['ceiling_height'].unique()

Уникальных значений стало намного меньше, с ними будет проще работать дальше. Все неправдоподобные значения были переделаны в нормальные

In [ ]:
#Проверим, нет ли отрицательных знаений в столбцах airports_nearest, cityCenters_nearest, days_exposition, floor, floors_total,
#kitchen_area, last_price, living_area, parks_around3000, parks_nearest, ponds_around3000, ponds_nearest, rooms, total_area, total_images
#for airports_nearest, cityCenters_nearest, days_exposition, floor, floors_total, kitchen_area, last_price, living_area, parks_around3000, parks_nearest, ponds_around3000, ponds_nearest, rooms, total_area, total_images in data:
    

print(data['airports_nearest'].sort_values().head())
print(data['cityCenters_nearest'].sort_values().head())
print(data['days_exposition'].sort_values().head())
print(data['floor'].sort_values().head())
print(data['floors_total'].sort_values().head())
print(data['kitchen_area'].sort_values().head())
print(data['last_price'].sort_values().head())
print(data['living_area'].sort_values().head())
print(data['parks_around3000'].sort_values().head())
print(data['parks_nearest'].sort_values().head())
print(data['ponds_around3000'].sort_values().head())
print(data['ponds_nearest'].sort_values().head())
print(data['rooms'].sort_values().head())
print(data['total_area'].sort_values().head())
print(data['total_images'].sort_values().head())

Отрицательных значений нет, работаем дальше. 
Вызывает сомнение существование одноэтажного дома, но всё возможно.
В столбцах living_area и rooms есть аномальные значения, скорее всего они вызваны тем, что квартира является студией. Ведь у неё нет комнат и жилая площадь может быть маленькой (например, все метры ушли на кухню). Также имеет место быть вариант продажи доли.

In [ ]:
#Посмотрим наличие дубликатов в датафрейме
data.duplicated().sum()

In [ ]:
#Выведем общую информацию датафрейма
data.info()
data.describe()

In [ ]:
#Заменим тип данных в столце last_price на целочисленный
data['last_price'] = data['last_price'].astype('int')
#Проверим результат
data['last_price'].head()

**Вывод:**
Данные обработаны насколько возможно. 
* Пропуски заменены подходящими по логике значениями
* Навания населённых пунктов приведены в нормальный вид
* Изменены типы данных


### Посчитайте и добавьте в таблицу новые столбцы

In [ ]:
#Добавим столбец с ценой квадратного метра и приведём столбец к типу int
data['one_met_price'] = data['last_price'] / data['total_area']
data['one_met_price'] = data['one_met_price'].astype(int)
#Проверим результат
print(data['one_met_price'])

In [ ]:
#Добавим столбец с днём недели публикации объявления
data['weekday'] = data['first_day_exposition'].dt.weekday
#Проверим результат
print(data['weekday'])

In [ ]:
#Добавим столбец с месяцем публикации объявлления и проверим результат
data['month'] = data['first_day_exposition'].dt.month
print(data['month'])

In [ ]:
#Добавим год публикации объявления и проверим результат
data['year'] = data['first_day_exposition'].dt.year
print(data['year'])

In [ ]:
#Напишем функцию категориации этажей и добавим столбец
def floor_cat(row):
    floors_total = row['floors_total']
    floor = row['floor']
    if floor == 1:
        return 'первый'
    elif floor == floors_total:
        return 'последний'
    elif 1 < floor < floors_total:
        return 'другой'
    
data['floor_category'] = data.apply(floor_cat, axis=1)
#Посмотрим на результат
print(data['floor_category'])

In [ ]:
#Добавим столбец с расстоянием до центра города в километрах
data['center_distance'] = data['cityCenters_nearest'] / 1000
#Округлим до целых значений
data['center_distance'] = data['center_distance'].round()
#Проверим результат
print(data['center_distance'])

**Вывод:**
Столбцы добавлены, данных для анализа стало больше

### Проведите исследовательский анализ данных

In [ ]:
#Построим гистограмму для изучения общей площади и выведем данные столбца
data.plot(y='total_area', kind='hist', bins=100, grid=True, figsize=(5, 3), range=(12, 150))
data['total_area'].describe()

**Вывод:** Самое пиковое значение на гистограмме у квартир с площадью 45 квадратных метров. Много квартир с площадью от 30 до 45 метров квадратных. Также есть довольно объёмный участок от 45 до 60 квадратных метров. Потом плотность идёт на спад. У гистограммы есть хвост справа. В данных есть выбросы, но на среднее они не особо влияют.

In [ ]:
#Построим гистограмму для изучения жилой площади и выведем данные столбца
data.plot(y='living_area', kind='hist', bins=100, grid=True, figsize=(5, 3), range=(0, 100))
data['living_area'].describe()

**Вывод:** Самое пиковое значение у жилой площади на отметке 17 кв. метров. Значения в основном распределяются от 14 до 50 кв. метров. Между этими значениями есть участок, в котором отражено мало значений. Квартир с жилой площадью от 21 о 25 кв. метров на рынке немного. Далее плотность идёт на спад. У гистограммы есть длинный хвост справа. Это говорит о том,что квартир с жилой площадью больше 55 кв. метров очень мало.

In [ ]:
#Построим гистограмму для изучения площади кухни и выведем данные столбца
data.plot(y='kitchen_area', kind='hist', bins=100, grid=True, figsize=(5, 3), range=(0, 60))
data['kitchen_area'].describe()

**Вывод:** Основная масса значений располагается в диапазоне от 5 до 15 кв. метров. Больше всего квартир с кухней 7 кв. метров. У гистограммы есть хвост справа, показывающий, что кухонь с метражом больше 15 кв. метров очень мало.

In [ ]:
#Построим гистограмму для изучения цены объекта и выведем данные столбца
data.plot(y='last_price', kind='hist', bins=100, grid=True, figsize=(5, 3), range=(0, 15000000))
data['last_price'].describe()

**Вывод:** Гистограмма имеет вид пирамиды с небольшим хвостом справа. Наибольшее количество квартир продаётся за 4 млн.р. Квартиры в основном продаются в диапазоне от 2 млн. р. до 6.5 млн.р. Мало квартир продаётся дороже, чем 7 млн.р.

In [ ]:
#Построим гистограмму для изучения количества комнат и выведем данные столбца
data.plot(y='rooms', kind='hist', bins=25, grid=True, figsize=(5, 3), range=(0, 10))
data['rooms'].describe()

**Вывод:** В данных есть выбросы, но на среднее это не влияет. На рынке больше всего однокомнатных и двухкомнатных квартир. Потом оидут трёхкомнатные. Четырёхкомнатных совсем мало и редко когда встречаются квартиры с пятью или больше комнатами. Студий тоже довольно мало.

In [ ]:
#Построим гистограмму для изучения высоты потолков и выведем данные столбца
data.plot(y='ceiling_height', kind='hist', bins=10, grid=True, figsize=(5, 3), range=(2.5, 3.5))
data['ceiling_height'].describe()

**Вывод:** В основном на рынке у квартир потолки от 2.6 до 2.7 метров. Редко встречаются потолки ниже 2.6 метров и совсем реко выше 2.7 метров.

In [ ]:
#Построим гистограмму для изучения этажа квартиры и выведем данные столбца
data.plot(y='floor', kind='hist', bins=50, grid=True, figsize=(5, 3), range=(1, 30))
data['floor'].describe()

**Вывод:** Большая часть квартир находится в промежутке от 1 до 5 этажа. Потом количество квартир идёт на спад. Выше 12 этажа продаётся мало квартир. Возможно, квартиры во многоэтажках продаются реже.

In [ ]:
#Построим гистограмму для изучения типа этажа квартиры и выведем данные столбца
data['floor_category'].hist(bins=5, grid=True, figsize=(5, 3))
data['floor_category'].describe()

**Вывод:** Меньше всего квартир продаются на 1 этаже. Неалеко ушли картиры на последнем этаже. Чаще всего продатся квартиры в середине дома.

In [ ]:
#Построим гистограмму для изучения общего количества этажей в доме и выведем данные столбца
data.plot(y='floors_total', kind='hist', bins=50, grid=True, figsize=(5, 3), range=(1, 30))
data['floors_total'].describe()

**Вывод:** На рынке больше всего квартир продаётся в пятиэтажных домах. Можно предположить, что пятиэтажек построено намного больше, чем домов с другим количеством этажей. Недалеко от них ушли девятиэтажные дома, их тоже довольно много. Домов с другим количеством этажей не так много. Среди них выделяются 4-х, 10-ти, 12-ти, 16-ти и 25-ти этажные дома. Если рассматривать их отдельно от пятиэтажек и девятиэтажек, то квартир на продажу в тех домах примерно равное количество.

In [ ]:
#Построим гистограмму для изучения расстояния до центра города в метрах и выведем данные столбца
data.plot(y='cityCenters_nearest', kind='hist', bins=50, grid=True, figsize=(5, 3), range=(0, 30000))
data['cityCenters_nearest'].describe()

**Вывод:** Большинство квартир находится на расстоянии от 8000 до 17000 м от центра. Есть довольно внушительное количество квартир, расположенных на расстоянии от 4000 до 6000 м от центра. В остальных диапазонах квартир не так много.

In [ ]:
#Построим гистограмму для изучения расстояния до ближайшего аэропорта и выведем данные столбца
data.plot(y='airports_nearest', kind='hist', bins=50, grid=True, figsize=(5, 3), range=(5000, 70000))
data['airports_nearest'].describe()

**Вывод:** В основном продаваемые квартиры находятся в диапазоне от 10000 до 55000 м до аэропорта. До 10000м расположено мало квартир. Начиная с 55000м идёт хвост, которы означает, что есть квартиры, находящиеся длеко от аэропорта, но их не так много. Больше всего квартир находятся на расстоянии в 20000м от аэропорта.

In [ ]:
#Построим гистограмму для изучения расстояния до ближайшего парка и выведем данные столбца
data.plot(y='parks_nearest', kind='hist', bins=50, grid=True, figsize=(5, 3), range=(0, 5000))
data['parks_nearest'].describe()

**Вывод:** В данных очень много пропусков по этому столбцу, поэтому гистограмма может быть неточной. По имеющимся данным видно, что у большинства квартир парк находится на расстоянии в 500м. В основном парки удалены от квартир не дальше, чем на 900 метров. С удалённостью парков больше 1000м не так много квартир.

In [ ]:
#Построим гистограмму для изучения дня публикации объявления и выведем данные столбца
data.plot(y='weekday', kind='hist', bins=50, grid=True, figsize=(5, 3), range=(0, 6))
data['weekday'].describe()

**Вывод:** Преимущественно публикуются объявления в будние дни. В выходныые публикуется почти в два раа меньше объявлений, чем в первые дни. В четверг подано больше всего объявлений, хоть разница с другими днями и не такая большая.

In [ ]:
#Построим гистограмму для изучения месяца публикации объявления и выведем данные столбца
data.plot(y='month', kind='hist', bins=50, grid=True, figsize=(5, 3), range=(1, 12))
data['month'].describe()

**Вывод:** Больше всего публикаций было в феврале, марте, апреле и ноябре. Меньше всего публикаций было в мае.

In [ ]:
#Изучим, как быстро продавались квартиры
#Построим гистограмму по столбцу days_exposition
data.plot(y='days_exposition', kind='hist', bins=100, grid=True, figsize=(5, 3), range=(1, 300))
#Посчитаем среднее и медиану
print('Среднее значение равно:', data['days_exposition'].mean())
print('Медиана равна:', data['days_exposition'].median())

**Вывод:** Самое пиковое значение у отметки в 45 дней. Чуть поменьше - 60 дней. Некоторые объекты продаются более 300 дней, но тут нужно учесть, что некоторые объявления забывают снимать с публикации после продажи, и объявление так и висит многие месяцы. Среднее время продажи составляет 180 дней. Медианное - 95 дней. Судя по гистограмме быстрыми продажами можно посчитать диапазон от 1 до 60 дней. Долгими - диапазон от 200 и выше. Следует отметить, что на сайте Яндекс.Недвижимость объявления автоматически акрываются каждые 45 и 60 дней, что искажает данные. Можно было бы их удалить, но тогда могут потеряться данные о реальной продаже в эти дни.

In [ ]:
#Изучим зависимость полной стоимости от общей площади
total_area_pt = data.pivot_table(index='total_area', values='last_price', aggfunc=['mean', 'count', 'median'])
total_area_pt.columns = ['mean', 'count', 'median']
total_area_pt.plot(y='median', style='o')
total_area_pt.sort_values('median', ascending=False)

print(data['total_area'].corr(data['last_price']))

**Вывод:** Цена ощутимо зависит от общей площади. Корреляция Пирсона равна 0.6. Это доказывает, что чем больше площадь квартиры, тем выше цена за эту квартиру.

In [ ]:
#Изучим зависимость стоимости от жилой площади
living_area_pt = data.pivot_table(index='living_area', values='last_price', aggfunc=['mean', 'count', 'median'])
living_area_pt.columns = ['mean', 'count', 'median']
living_area_pt.plot(y='median', style='o')
living_area_pt.sort_values('median', ascending=False)

print(data['living_area'].corr(data['last_price']))

**Вывод:** Корреляция Пирсона равна 0.5. Значит, метраж жилой площади влияет на общую стоимость квартиры. Чем больше площадь, тем больше стоимость квартиры.

In [ ]:
#Изучим зависимость стоимости от площади кухни
kitchen_area_pt = data.pivot_table(index='kitchen_area', values='last_price', aggfunc=['mean', 'count', 'median'])
kitchen_area_pt.columns = ['mean', 'count', 'median']
kitchen_area_pt.plot(y='median', style='o')
kitchen_area_pt.sort_values('median', ascending=False)

print(data['kitchen_area'].corr(data['last_price']))

**Вывод:** Корреляция Пирсона составляет также 0.5. Зависимость стоимости есть, чем выше метраж кухни, тем выше стоимость квартиры. Хотя из графика видно, что это единичные случаи. Больше всего скопление в диапазоне от 0 до 55 кв. метров. И цена там не сильно различается.

In [ ]:
#Изучим зависимость стоимости от количества комнат
rooms_pt = data.pivot_table(index = 'rooms', values = 'last_price', aggfunc = ['mean', 'count', 'median'])
rooms_pt.columns = ['mean', 'count', 'median']
rooms_pt.query('count > 50').plot(y = 'median')

rooms_pt.query('count > 50').sort_values('median', ascending = False)

data['rooms'].corr(data['last_price'])

**Вывод:** Зависимость есть. С увеличением числа комнат, стоимость квартиры растёт.

In [ ]:
#Изучим зависимость стоимости от этажа, на котором расположена квартира (первый, последний, другой)
floor_category_pt = data.pivot_table(index = 'floor_category', values = 'last_price', aggfunc = ['mean', 'count', 'median'])
floor_category_pt.columns = ['mean', 'count', 'median']
floor_category_pt.plot(y = 'median')
floor_category_pt

**Вывод:** Из графика видно, что у квартир, расположенных на первом этаже, самая низкая цена. Самая высокая - у квартир, расположенных между первым и последним этажами. Цена на квартиры на посленем этаже не намного меньше цены на квартиры в середине дома. Цена зависит от этажа, если это первый или последний, осальные этажи будут примерно в одной ценовой категории.

In [ ]:
#Изучим зависимость стоимости от дня размещения
weekday_pt = data.pivot_table(index = 'weekday', values = 'last_price', aggfunc = ['mean', 'count', 'median'])
weekday_pt.columns = ['mean', 'count', 'median']
weekday_pt.plot(y = 'median')

weekday_pt.sort_values('median', ascending = False)

**Вывод:** Из графика можно сделать вывод, что цена зависит от дня размещения объявления. Во вторник публикуются самые дорогие объявления. К концу недели цена на квартиры идёт на спад. В воскресенье устанавливается самая низкая цена. Хотя разница между самой высокой и самой низкой ценой не такая большая. Это связано с тем, что в выходные публикуется почти в два раза меньше объявлений. 

In [ ]:
#Изучим зависимость стоимости от месяца размещения
month_pt = data.pivot_table(index = 'month', values = 'last_price', aggfunc = ['mean', 'count', 'median'])
month_pt.columns = ['mean', 'count', 'median']
month_pt.plot(y = 'median')

month_pt.sort_values('median', ascending = False)

**Вывод:** Самая высокая цена у объявлений, опубликованных в апреле, самая низкая - в июне. С начала года до апреля цена растёт, потом происходит падение. Возможно это связано с началом летнего сезона. Потом цена находится примерно в одном диапазоне. Повышается в нечётные месяцы, понижается в чётные.

In [ ]:
#Изучим зависимость стоимости от года размещения
year_pt = data.pivot_table(index = 'year', values = 'last_price', aggfunc = ['mean', 'count', 'median'])
year_pt.columns = ['mean', 'count', 'median']
year_pt.plot(y = 'median')

year_pt.sort_values('median', ascending = False)

**Вывод:** В 2014 году было подано меньше всего объявлений, этот год мы не будем учитывать. В 2017 и 2018 году было больше всего объявлений(8000 против 2000 в остальные годы), но медианная цена у них меньше всего. В 2016 году подано 2000 объявлений, а медианная цена совпадает с 2018 годом. Самая высокая цена была в 2015 и 2019 годах. И этого можно сделать вывод, что год подачи объявления влияет на цену, а точнее мировая обстановка на это влияет.

In [ ]:
#Изучим зависимость площади от года подачи объявления
area_pt = data.pivot_table(index = 'year', values = 'total_area', aggfunc = ['mean', 'count', 'median'])
area_pt.columns = ['mean', 'count', 'median']
area_pt.plot(y = 'median')

area_pt.sort_values('median', ascending = False)

**Вывод:** С каждым годом площадь продаваемых квартир уменьшалась, только в 2019 году немного подросла по сравнени с предыущим годом. А так так цена зависит от площади, то и стоимость объектов снижалась соответственно. Зависимости цены от года нет, всё идёт от площади объектов.

In [ ]:
#Посчитаем среднюю цену одного квадратного метра в 10 населённых пунктах с наибольшим числом объявлений и 
#выделим населённые пункты с самой высокой и низкой стоимостью квадратного метра
local_pt = data.pivot_table(index = 'locality_name', values = 'one_met_price', aggfunc=['count', 'mean'])
local_pt.columns = ['count', 'mean']
local_pt = local_pt.sort_values('count', ascending = False).head(10)
print(local_pt)
#самая высокая стоимость
max_price = local_pt[local_pt['mean']==local_pt['mean'].max()]
#самая низкая стоимость
min_price = local_pt[local_pt['mean']==local_pt['mean'].min()]

display(max_price)
display(min_price)

In [ ]:
#Посчитаем медианную цену одного квадратного метра в 10 населённых пунктах с наибольшим числом объявлений и 
#выделим населённые пункты с самой высокой и низкой стоимостью квадратного метра
local_pt = data.pivot_table(index = 'locality_name', values = 'one_met_price', aggfunc=['count', 'median'])
local_pt.columns = ['count', 'median']
local_pt = local_pt.sort_values('count', ascending = False).head(10)
print(local_pt)
#самая высокая стоимость
max_price = local_pt[local_pt['median']==local_pt['median'].max()]
#самая низкая стоимость
min_price = local_pt[local_pt['median']==local_pt['median'].min()]

display(max_price)
display(min_price)

**Вывод:** Самая высокая цена за один кв. метр на жильё в Санкт-Петербурге. Самая низкая - в Выборге.

In [ ]:
#Выделим квартиры в Санкт-Петербурге с помощью столбца locality_name и вычислим среднюю цену каждого километра
data_spb = data.copy()
data_spb = data_spb.query('locality_name == "Санкт-Петербург"')
data_spb.loc[:, 'center_distance'] = (data_spb.loc[:, 'cityCenters_nearest']/1000).round()
data_spb_km = data_spb.pivot_table(index = 'center_distance', values = 'one_met_price', aggfunc = 'mean')
data_spb_km.plot(grid = True, title = 'Цена за километр')
plt.show()

**Вывод:** Из графика видно, что цена на квартиры увеличивается по мере приближения к центру. Есть 2 пика на отметках в 27 и 7 км. Может быть эти объекты намного лучше своих конкурентов, находящихся на том же расстоянии от центра, от того и цена у них выше. Другой вариант - собственники квартир установили слишком высокие цены просто потому что захотели. Начиная с отметки в 3км, цена начинает расти очень быстро. Получается, чем ближе к центру находится квартира, тем она будет дороже.

### Общий вывод

В ходе работы было сделано:
* Определены пропуски и обозначены варианты их заполнения
* Пропуски заменены на значения там, где это было возможно
* Определены и исправлены аномалии в данных
* Названия населённых пунктов приведены к общему виду
* Изменены типы данных там, где это было необходимо
* Добавлены столбцы в датасет для лучшего изучения картины
* Проведён анализ имещихся данных, в котором выяснилось, что:
    + Цена квартиры зависит в основном от метража, удалённости от центра и немного от этажа
    + Квартиры в среднем продаются 180 дней, но есть квартиры, которые продаются 200 дней и дольше
    + В Санкт-Петербурге самая высокая цена на квартиры
    + В Выборге самая низкая цена на квартиры
* Даны ответы на заданные вопросы